In [1]:
import sys
import os
import numpy as np
# Get the current directory of the notebook
notebook_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
# Add the 2nd level parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(parent_dir, '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import bemt_param_fitter
import inflow_model.propeller_lookup_table as propeller_lookup_table

Prepare data for fitting. 

In [2]:
import data_factory

fitting_subfolder = "wind_near_wall_bemt_fitting"
# fitting_subfolder = "wind_near_wall_bemt_unit_test"
factory = data_factory.FittingFactory()
# data_list = data_factory.generate_data_list(fitting_subfolder, '.pkl')
data_list = data_factory.generate_data_list(fitting_subfolder, '.csv')
print(f"Fitting Data list:")
for data in data_list:
    print(data)
datasets = factory.prepare_datasets(data_list)

fitting_subfolder = "wind_near_wall_bemt_fitting_validation"
factory = data_factory.FittingFactory()
# data_list = data_factory.generate_data_list(fitting_subfolder, '.pkl')
data_list = data_factory.generate_data_list(fitting_subfolder, '.csv')
print(f"Validtion Data list:")
for data in data_list:
    print(data)
datasets_validation = factory.prepare_datasets(data_list)

Fitting Data list:
wind_near_wall_bemt_fitting\test_wind_near_wall_x0_0_y0_0_z0_0_d100_4_fitting.csv
wind_near_wall_bemt_fitting\test_wind_near_wall_x0_0_y0_0_z5_0_d100_4_fitting.csv
wind_near_wall_bemt_fitting\test_wind_near_wall_x0_0_y0_0_zn5_0_d100_4_fitting.csv
wind_near_wall_bemt_fitting\test_wind_near_wall_xn3_0_y0_0_z0_0_d100_4_fitting.csv
wind_near_wall_bemt_fitting\test_wind_near_wall_xn3_0_y0_0_z5_0_d100_4_fitting.csv
wind_near_wall_bemt_fitting\test_wind_near_wall_xn3_0_y0_0_zn5_0_d100_4_fitting.csv
Validtion Data list:
wind_near_wall_bemt_fitting_validation\test_wind_near_wall_x0_0_y0_0_z10_0_d0_5_validation_no_bemt.csv
wind_near_wall_bemt_fitting_validation\test_wind_near_wall_x0_0_y0_0_z1_0_d0_5_validation_no_bemt.csv
wind_near_wall_bemt_fitting_validation\test_wind_near_wall_x0_0_y0_0_z4_0_d0_5_validation_no_bemt.csv
wind_near_wall_bemt_fitting_validation\test_wind_near_wall_x0_0_y0_0_zn10_0_d0_5_validation_no_bemt.csv
wind_near_wall_bemt_fitting_validation\test_wind_nea

Start fitting.

In [ ]:
fitter = bemt_param_fitter.BemtParamFitter()
fitter.fit_params(datasets, np.array([5.31194203, 1.46627105, 1.72517079, 0.45174155]), is_fine_tune=False)

To test the fitted params, one can directly apply the fitted solution to BEMT model to generate result to compare with the label. Use sparse samples as this is computationally expensive. 

In [10]:
fitter = bemt_param_fitter.BemtParamFitter()
x = np.array([5.3, 1.7, 1.8, np.radians(20.6)]) # ground truth
fitter.adjust_resolution(True)
lookup_table = propeller_lookup_table.PropellerLookupTable.Reader("apc_8x6_fitted2")
# lookup_table = propeller_lookup_table.PropellerLookupTable.Reader("apc_8x6_with_trail_refine")
fitter.can_log_and_display_data = False
fitter.get_loss(x, datasets[:5], lookup_table=lookup_table, is_using_lookup_table=True)

[PropellerLookupTable] Reading data from ..\..\inflow_model\lookup_table\apc_8x6_fitted2.yaml
Current loss: 8.592371974062772


8.592371974062772

To do a thorough test, generate a lookup table using inflow_model.propeller_lookup_table_users_guide.ipynb and come back here to check the loss over all sammple data.

In [ ]:
fitter.make_lookup_table()

Append the residual force calculated based on the lookup table to the end of the dataset. In the next step, this augmented dataset will be used for network training.

In [11]:
for dataset in datasets:
    print(f"Filling residual force column of {dataset}")
    fitter.make_residual_force_columns(dataset, lookup_table)


Filling residual force column of <data_factory.FittingDataset object at 0x000002D164DBC190>
Filling residual force column of <data_factory.FittingDataset object at 0x000002D12DD09BA0>
Filling residual force column of <data_factory.FittingDataset object at 0x000002D164D6C070>
Filling residual force column of <data_factory.FittingDataset object at 0x000002D12B3CA290>
Filling residual force column of <data_factory.FittingDataset object at 0x000002D12B639330>
Filling residual force column of <data_factory.FittingDataset object at 0x000002D164DBD4E0>


Similarly, append the residual force to validation dataset as well for validation after training.

In [12]:
for dataset in datasets_validation:
    print(f"Filling residual force column of {dataset}")
    fitter.make_residual_force_columns(dataset, lookup_table)

Filling residual force column of <data_factory.FittingDataset object at 0x000002D164CD1690>
Filling residual force column of <data_factory.FittingDataset object at 0x000002D164DBFB50>
Filling residual force column of <data_factory.FittingDataset object at 0x000002D12EDE6BC0>
Filling residual force column of <data_factory.FittingDataset object at 0x000002D12B3CA6B0>
Filling residual force column of <data_factory.FittingDataset object at 0x000002D164D6E7D0>
Filling residual force column of <data_factory.FittingDataset object at 0x000002D164D6D600>
Filling residual force column of <data_factory.FittingDataset object at 0x000002D12B6392A0>
Filling residual force column of <data_factory.FittingDataset object at 0x000002D1259CE410>
Filling residual force column of <data_factory.FittingDataset object at 0x000002D12B639720>
Filling residual force column of <data_factory.FittingDataset object at 0x000002D164DBEF50>
Filling residual force column of <data_factory.FittingDataset object at 0x000002